<a href="https://colab.research.google.com/github/ARBML/adawat/blob/main/notebooks/ALUE_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ALUE (Arabic Language Understanding Evaluation) Benchmark:
## Universal Sentence Encoder (USE) Baseline:
### SemEval-2018 Task 1 -- Affect in Tweets: Emotion Classification Task

In this notebook, we will walk you through the process of reproducing the Universal Sentence Encoder baseline for the E-c task.

#### Loading Required Modules
First off, we start by loading the needed python libraries.

In [ ]:
! pip install -q tensorflow-text

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd
import tensorflow_text
from tensorflow import keras
from sklearn.metrics import jaccard_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

#### Loading Data
Using pandas, we can load and inspect the training, validation, and testing datasets as follows:

* Data can be downloaded from here: https://www.alue.org/tasks => SemEval-2018 Task 1 - Affect in Tweets: Emotion Classification task

In [ ]:
df_train = pd.read_csv("/content/2018-E-c-Ar-train.txt", sep="\t")
df_dev = pd.read_csv("/content/2018-E-c-Ar-dev.txt", sep="\t")
df_test = pd.read_csv("/content/emotion_no_labels_v1.0.tsv", sep="\t")

Below we list the 5 first entries in the training data.

In [ ]:
df_train.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-Ar-00259,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,1,0,0,1,0,0,0,1,1,0,0
1,2018-Ar-02696,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...,0,0,0,0,1,1,1,0,0,0,0
2,2018-Ar-03596,البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متو...,0,0,0,0,0,1,1,0,0,0,0
3,2018-Ar-02999,مومعقول اللي قاعد يصير فيني هالايام يارب ماينت...,0,0,0,1,0,0,0,0,0,0,0
4,2018-Ar-02716,انا اكثر شخص متناقض بداخلي حب وكره وامل وقنوط ...,1,0,0,0,0,0,0,0,1,0,0


And the 5 first entries in the development data.

In [ ]:
df_dev.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-Ar-00289,باقي ١٠ دقايق واخلص حلقة سكول الثانيه بس النت ...,1,0,1,0,1,0,0,1,0,0,0
1,2018-Ar-02519,معاناة لما يكون دايماً إحساسك بمحله .....,0,0,0,0,0,0,0,0,1,0,0
2,2018-Ar-01952,لو فيه جائزه اكثر موسوسه تخاف من الامراض فزت ب...,0,0,0,1,0,0,0,1,0,0,0
3,2018-Ar-02912,ما يستفز راحة البال إلا الذكريات ☹️ 💔,1,0,0,0,0,0,0,0,1,0,0
4,2018-Ar-02756,القلب ياوقت والنفس ماهي مرتاحه ونبض القلب ماهو...,0,0,0,1,0,0,0,0,1,0,0


And last but not least, the first 5 entries in the test data.

In [ ]:
df_test.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,17439,عيونك تهبل عيونك هي الآلئ اللي تظوي حياتي ما ا...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,10196,كم هو موجع في بعض الاحيان ان يعمل الانسان با ط...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,17470,انا هولندي عندي مناعة عخيبات الامل... ما في شي...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,16262,احا المانيا خرجت,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,13597,وش الي صار متى سجلت الارجنتين,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


#### Model Preparation
We start by setting the randomisation seed:

In [ ]:
tf.random.set_seed(123)

Next we load the Universal Sentence Encoder (WARNING: This will download and cache a huge model of around 1 GB in size)

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

Then we define the input and output to the model:

In [ ]:
sentence = keras.Input(shape=512, name='sentence')
label = keras.Input(shape=(11,), name='label')

This is followed by defining the structure of the network:

In [ ]:
logits = keras.layers.Dense(512, activation=tf.nn.sigmoid)(sentence)
logits = keras.layers.Dense(512, activation=tf.nn.sigmoid)(logits)
logits = keras.layers.Dense(11, activation=tf.nn.sigmoid)(logits)

Then we construct and compile the model:

In [ ]:
model = keras.Model(sentence, outputs=logits)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Model Training
First we perpare the inputs and outputs to be fed to the model during training:

In [ ]:
X_train = embed(df_train["Tweet"])
Y_train = df_train[df_train.columns[2:]]
X_dev = embed(df_dev["Tweet"])
Y_dev = df_dev[df_dev.columns[2:]]

Next we fit the data:

In [ ]:
model.fit(X_train,
          Y_train,
          epochs=5,
          batch_size=32,
          validation_data = (X_dev, Y_dev))

Epoch 1/5
72/72 [==============================] - 2s 13ms/step - loss: 0.4792 - accuracy: 0.2827 - val_loss: 0.4849 - val_accuracy: 0.0650
Epoch 2/5
72/72 [==============================] - 1s 8ms/step - loss: 0.4571 - accuracy: 0.3077 - val_loss: 0.4580 - val_accuracy: 0.3692
Epoch 3/5
72/72 [==============================] - 0s 7ms/step - loss: 0.4249 - accuracy: 0.3867 - val_loss: 0.4149 - val_accuracy: 0.4410
Epoch 4/5
72/72 [==============================] - 1s 7ms/step - loss: 0.3924 - accuracy: 0.4377 - val_loss: 0.4005 - val_accuracy: 0.4872
Epoch 5/5
72/72 [==============================] - 0s 7ms/step - loss: 0.3839 - accuracy: 0.4653 - val_loss: 0.3997 - val_accuracy: 0.5419


We calculate the Jaccard similarity score for the development set as follows:

In [ ]:
jaccard_score(model.predict(X_dev)>0.5, Y_dev, average="macro")

19/19 [==============================] - 0s 4ms/step


0.21492762532440446

#### Submission Preperation
We perpare the features for each testset instance as follows:

In [ ]:
X_test = embed(df_test["Tweet"])

Then we predict the labels for each:

In [ ]:
predictions = (model.predict(X_test)>0.5).astype(int)

32/32 [==============================] - 0s 3ms/step


We perpare the predictions as a pandas dataframe:

In [ ]:
df_preds = pd.DataFrame(data=predictions, columns=df_train.columns[2:].tolist(), index=df_test["ID"])
df_preds.reset_index(inplace=True)

We explore the prediction dataframe:

In [ ]:
df_preds.head()

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,17439,0,0,0,0,0,0,0,0,1,0,0
1,10196,1,0,0,0,0,0,0,0,0,0,0
2,17470,1,0,0,0,0,0,0,0,1,0,0
3,16262,1,0,0,0,0,0,0,0,0,0,0
4,13597,1,0,1,0,0,0,0,0,1,0,0


In the final step, we save the predictions as required by the competition guidelines.

In [ ]:
if not os.path.exists("predictions"):
    os.mkdir("predictions")
df_preds.to_csv("./predictions/E_c.tsv", index=False, sep="\t")

To make sure the extracted predictions are readable:

In [ ]:
extracted_preditctions = pd.read_csv("/content/predictions/E_c.tsv", sep="\t")
extracted_preditctions

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,17439,0,0,0,0,0,0,0,0,1,0,0
1,10196,1,0,0,0,0,0,0,0,0,0,0
2,17470,1,0,0,0,0,0,0,0,1,0,0
3,16262,1,0,0,0,0,0,0,0,0,0,0
4,13597,1,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,6877,1,0,1,0,0,0,0,0,0,0,0
996,10056,1,0,0,0,0,0,0,0,1,0,0
997,16364,1,0,0,0,0,0,0,0,0,0,0
998,5406,1,0,0,0,0,0,0,0,0,0,0


\* __Disclaimer:__ <br> This notebook is a modified version from: 
https://github.com/Alue-Benchmark/alue_baselines/blob/master/notebooks/USE/E-c.ipynb